## 模型

In [ ]:
from transformers import BertModel, AutoModel

In [ ]:
# 优雅地下载 huggingface 模型
# !pip install huggingface_hub
from huggingface_hub import snapshot_download
snapshot_download(repo_id='Helsinki-NLP/opus-mt-zh-en', cache_dir="models/opus-mt-zh-en/", local_dir_use_symlinks=False)

In [ ]:
model = BertModel.from_pretrained("bert-base-cased")  # 自动下载模型权重，默认保存到 ~/.cache/huggingface/transformers
# model = AutoModel.from_pretrained("bert-base-cased")  # 我们也可以通过 HF_HOME 环境变量来指定缓存目录

# 注意，在大部分情况下，我们都应该使用 AutoModel 来加载模型
# 这样如果我们想要使用另一个模型（比如把 BERT 换成 RoBERTa），只需修改 checkpoint，其他代码可以保持不变

# 保存模型
model.save_pretrained("bert-base-cased")  # 保存到当前目录下

In [ ]:
# 除了从 HF Hub 加载模型，我们也可以从本地加载模型
model = BertModel.from_pretrained("models/bert-base-chinese")
# config.json: 模型配置文件, 存储模型结构参数，例如 Transformer 层数、特征空间维度等；
# pytorch_model.bin：模型权重文件，存储模型所有的权重参数；

## 分词器
### 基本操作
由于神经网络模型不能直接处理文本，因此我们需要先将文本转换为数字，这个过程被称为编码 (Encoding)，其包含两个步骤：

1. 使用分词器 (tokenizer) 将文本按词、子词、字符切分为 tokens；
2. 将所有的 token 映射到对应的 token ID。

In [ ]:
from transformers import AutoTokenizer

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenizer.save_pretrained("models/bert-base-cased/")
# special_tokens_map.json: 存储特殊 token 对应的 id，例如 [CLS]、[SEP]、[PAD] 等；
# tokenizer_config.json: 分词1器配置文件，存储分词器的配置参数，例如是否区分大小写、是否处理未知词等；
# vocab.txt: 词表文件，一行一个 token，行号就是对应的 token id；

In [ ]:
# 1. 分词
tokenizer = AutoTokenizer.from_pretrained("models/bert-base-cased/")
sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)
print(tokens)
# BERT 分词器采取的是字词切分策略，它会不断切分词语直到获得词表中的 token
# 例如 “transformer” 会被切分为 “transform” 和 “##er”。

# 2. 将 tokens 转换为 token id
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

# 或者使用 encode() 方法直接将文本转换为 token id, 该方法会自动添加 [CLS] 和 [SEP] 标记
ids = tokenizer.encode(sequence)
print(ids)

In [ ]:
# 在实际编码文本时，最常见的是直接使用分词器进行处理，这样不仅返回分词结果，还会输出模型需要的其他输入
# 例如 BERT 分词器还会自动在输入中添加 token_type_ids 和 attention_mask
tokenized_text = tokenizer(sequence)
print(tokenized_text)

In [ ]:
# 解码负责将 token id 转换为文本
# 注意，解码过程不是简单地将 token IDs 映射回 tokens，还需要合并那些被分为多个 token 的词语，即用 ##er 合并 “transform” 和 “##er”
decoded_string = tokenizer.decode([6627, 714, 3130, 1762, 2207, 5101, 4906, 2825, 1736])
print(decoded_string)

decoded_string = tokenizer.decode([101, 6627, 714, 3130, 1762, 2207, 5101, 4906, 2825, 1736, 102])
print(decoded_string)

# 如果使用 skip_special_tokens=True，那么解码过程会自动跳过特殊 token
decoded_string = tokenizer.decode([101, 6627, 714, 3130, 1762, 2207, 5101, 4906, 2825, 1736, 102], skip_special_tokens=True)
print(decoded_string)

In [ ]:
# 处理多段文本
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "So have I!"
]
max_length = 8
# 通过 max_length 参数来控制截断长度：
tokenized_inputs = tokenizer(sequences, padding="max_length", max_length=max_length, truncation=True, return_tensors="pt")
# 也可以使用 return_tensors="tf" 来返回 TensorFlow 张量
print(tokenized_inputs)

# **tokens: 将字典解包为关键字参数
output = model(**tokens)

### 处理长文本
注意，目前大部分 Transformer 模型只能接受长度不超过 512 或 1024 的 token 序列，因此对于长序列，有以下三种处理方法：
1. 使用一个支持长文的 Transformer 模型，例如 Longformer 和 LED（最大长度 4096）；
2. 设定最大长度 max_sequence_length 以截断输入序列：sequence = sequence[:max_sequence_length]。
3. 将长文切片为短文本块 (chunk)，然后分别对每一个 chunk 编码

### 添加新 token
1. add_tokens() 添加普通 token：参数是新 token 列表，如果 token 不在词表中，就会被添加到词表的最后
2. add_special_tokens() 添加特殊 token：参数是包含特殊 token 的字典，键值只能从 bos_token, eos_token, unk_token, sep_token, pad_token, cls_token, mask_token, additional_special_tokens 中选择

注意：特殊 token 的标准化 (normalization) 与普通 token 有一些不同，比如不会被小写。

In [ ]:
# add_tokens() 添加普通 token
num_added_toks = tokenizer.add_tokens(["new_token1", "my_new-token2"])
print("We have added", num_added_toks, "tokens")

# 为了防止 token 已经包含在词表中，我们可以预先对新 token 列表进行过滤
new_tokens = ["new_token1", "my_new-token2"]
new_tokens = set(new_tokens) - set(tokenizer.vocab.keys())
tokenizer.add_tokens(list(new_tokens))

In [ ]:
# add_special_tokens() 添加特殊 token
special_tokens_dict = {"cls_token": "[MY_CLS]"}

num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print("We have added", num_added_toks, "tokens")

assert tokenizer.cls_token == "[MY_CLS]"

### 调整 embedding 矩阵
向词表中添加新 token 后，必须重置模型 embedding 矩阵的大小，也就是向矩阵中添加新 token 对应的 embedding，这样模型才可以正常工作，将 token 映射到对应的 embedding。

In [ ]:
# 调整 embedding 矩阵
model.resize_token_embeddings(len(tokenizer))
print(model.embeddings.word_embeddings.weight.size())
# 在默认情况下，调整 embedding 矩阵后，新添加的 token 会被随机初始化

In [ ]:
# 更高级的初始化方式：初始化为已有 token 的值
descriptions = ['start of entity', 'end of entity']

# 将 [ENT_START] 的 embedding 初始化为 “start”、“of”、“entity” 三个 token 的平均值
with torch.no_grad():
    for i, token in enumerate(reversed(descriptions), start=1):
        tokenized = tokenizer.tokenize(token)
        print(tokenized)
        tokenized_ids = tokenizer.convert_tokens_to_ids(tokenized)
        new_embedding = model.embeddings.word_embeddings.weight[tokenized_ids].mean(axis=0)
        model.embeddings.word_embeddings.weight[-i, :] = new_embedding.clone().detach().requires_grad_(True)
print(model.embeddings.word_embeddings.weight[-2:, :])


### 快速分词器

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("models/bert-base-cased")
example = "My name is Sylvain and I work at Hugging Face in Brooklyn."
encoding = tokenizer(example)
# 获取切分出的 token
print(encoding.tokens())

In [ ]:
# 追踪映射
# 在上面的例子中，索引为 5 的 token 是 “##yl”，它是词语 “Sylvain” 的一个部分，因此在映射回原文时不应该被单独看待
# 我们可以通过 word_ids() 函数来获取每一个 token 对应的词语索引：

print(encoding.word_ids())

# 可以看到，特殊 token [CLS] 和 [SEP] 被映射到 None，其他 token 都被映射到对应的来源词语
# 这可以为很多任务提供帮助，例如对于序列标注任务，就可以运用这个映射将词语的标签转换到 token 的标签

In [ ]:
print(example[11:19])  # 根据开始位置和结束位置获取人名

In [ ]:
# char_to_token() 函数可以将字符索引转换为 token 索引
start = encoding.char_to_token(11)
end = encoding.char_to_token(19)
print(start, end)
print(encoding.tokens()[start:end])